In [4]:
#from google.colab import files
#uploaded = files.upload()

#from google.colab import files
#uploaded = files.upload()

!wget "https://www.dropbox.com/s/ux6foch7s5duq4s/datav2.zip?dl=1" -O "datav2.zip"
!unzip "datav2.zip"
!ls

!echo "gg ez"

--2018-04-19 09:51:46--  https://www.dropbox.com/s/ux6foch7s5duq4s/datav2.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.1, 2620:100:6031:1::a27d:5101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/AefzfpRPbdgOoT6MRZYsoHJ2XVUuqcZgD9pmwpQiRkHMVvdTYa6Rh3b6JKIpPmIC/file?dl=1 [following]
--2018-04-19 09:51:48--  https://dl.dropboxusercontent.com/content_link/AefzfpRPbdgOoT6MRZYsoHJ2XVUuqcZgD9pmwpQiRkHMVvdTYa6Rh3b6JKIpPmIC/file?dl=1
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.81.6, 2620:100:601f:6::a27d:906
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.81.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 141703830 (135M) [application/binary]
Saving to: ‘datav2.zip’

datav2.zip          100%[===================>] 135.14M  11.9MB/s    in 13s     

20

In [5]:
from keras import backend as K
K.clear_session()


Using TensorFlow backend.


In [0]:
from IPython import embed
import keras
import numpy as np
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from keras.layers import Add, merge, Lambda
from keras import regularizers
from keras.models import Sequential, Model


vgg = VGG16(weights='imagenet', include_top=False, input_shape=(48,48, 3))

def my_init(initval):
  def custominit(shape, dtype=None):
    return initval
  return custominit
  
# opt: 
# opt["model"] = {"vgg", "wide_resnet"}
# opt["fc_after_cnn"] = {True,False}
# opt["fc_after_cnn_dim"] = _Integer_
# opt["comparing_method"] = {"dot","mmul"}
# opt["comparing_method_mmul_w_dim"] = _Integer_
# opt["comparing_method_mmul_use_original_target"] = {True,False}

def build_model(shape, w_reg, k, opt={}):
  # shape should be (k+1)x100x100x3 for some k
  # first is target, rest are candidates
  
  dim_x, dim_y, dim_depth = shape[1],shape[2],shape[3]
  
  input = Input(shape=shape)
  # hide k+1 headshots in one batch to tie weights
  x = Lambda(lambda i: K.tf.reshape(i, (-1, dim_x,dim_y,dim_depth)))(input)
  
  cnn = None
  if opt["model"] == "vgg":
    cnn = vgg_f
  elif opt["model"] == "wide_resnet":
    cnn = wide_resnet_f
  cnn_out = cnn(x, w_reg)
  
  # Flatten CNN
  flattened = Lambda(lambda i: K.tf.reshape(i, (-1, i.shape[1]*i.shape[2]*i.shape[3])))(cnn_out)
  x = flattened
  cnn_out_dim = x.shape[1].value
  
  print("CNN output size:", cnn_out_dim)
  
  if "fc_after_cnn" in opt and opt["fc_after_cnn"]:
    x = Dense(opt["fc_after_cnn_dim"], 
            activation="relu",
            kernel_initializer="glorot_normal", 
            kernel_regularizer=regularizers.l2(w_reg))(x)
    x = BatchNormalization()(x)
  dims = x.shape[1].value
  
  print("Embedding size:", dims)
  
  x = Lambda(lambda i: K.tf.reshape(i, [-1, k+1, dims]))(x)

  target = x[:,0,:] # [n, dims]
  candidates = x[:,1:,:] # [n, k, dims]  
  if opt["comparing_method"] == "dot":
    print("Doing dot product similarity")
    
    _x = K.tf.multiply(candidates, K.tf.reshape(target, [-1,1,dims]))
    _x = K.tf.reshape(_x, [-1, dims])
    x = Lambda(lambda i: _x)(x)
    x = Dense(1, kernel_initializer="glorot_normal", kernel_regularizer=regularizers.l2(w_reg))(x)
    x = Lambda(lambda i: K.tf.reshape(i, [-1, k]))(x)
    x = Activation("softmax")(x)
  elif opt["comparing_method"] == "mmul":
    print("Doing mmul weights prediction with shape:", [dims, opt["comparing_method_mmul_w_dim"]])
    
    # use original vs slimmed embedding
    if "comparing_method_mmul_use_original_target" in opt and opt["comparing_method_mmul_use_original_target"]:
      _c = Lambda(lambda i: K.tf.reshape(flattened, [-1,k+1, cnn_out_dim])[:,0,:])(x)
    else:
      _c = Lambda(lambda i: target)(x)
      
    c = Dense(dims * opt["comparing_method_mmul_w_dim"], 
            activation="relu",
            kernel_initializer="glorot_normal", 
            kernel_regularizer=regularizers.l2(w_reg))(_c)
    
    _w = K.tf.reshape(c, [-1, dims, opt["comparing_method_mmul_w_dim"]])
    _x = K.tf.reshape(K.tf.matmul(candidates, _w), [-1, k, opt["comparing_method_mmul_w_dim"]])
    _x = K.tf.reshape(_x, [-1, opt["comparing_method_mmul_w_dim"]])
    x = Lambda(lambda i: _x)(c)
    
    x = Dense(1, kernel_initializer="glorot_normal", kernel_regularizer=regularizers.l2(w_reg))(x)
    x = Lambda(lambda i: K.tf.reshape(i, [-1, k]))(x)
    x = Activation("softmax")(x)
  print(x.shape)
  return Model(inputs=input, outputs=x)
  

def vgg_f(x, w_reg):
  # first 3 groups of vgg
  x = Conv2D(64, 3, padding="same", activation="relu", 
             kernel_initializer=my_init(vgg.layers[1].weights[0]),
             bias_initializer=my_init(vgg.layers[1].weights[1]),
             kernel_regularizer=regularizers.l2(w_reg))(x)
  x = BatchNormalization()(x)
  x = Conv2D(64, 3, padding="same", activation="relu", 
             kernel_initializer=my_init(vgg.layers[2].weights[0]),
             bias_initializer=my_init(vgg.layers[2].weights[1]),
             kernel_regularizer=regularizers.l2(w_reg))(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D(2)(x)
  
  x = Conv2D(128, 3, padding="same", activation="relu", 
             kernel_initializer=my_init(vgg.layers[4].weights[0]),
             bias_initializer=my_init(vgg.layers[4].weights[1]),
             kernel_regularizer=regularizers.l2(w_reg))(x)
  x = BatchNormalization()(x)
  x = Conv2D(128, 3, padding="same", activation="relu", 
             kernel_initializer=my_init(vgg.layers[5].weights[0]),
             bias_initializer=my_init(vgg.layers[5].weights[1]),
             kernel_regularizer=regularizers.l2(w_reg))(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D(2)(x)
  
  x = Conv2D(256, 3, padding="same", activation="relu", 
             kernel_initializer=my_init(vgg.layers[7].weights[0]),
             bias_initializer=my_init(vgg.layers[7].weights[1]),
             kernel_regularizer=regularizers.l2(w_reg))(x)
  x = BatchNormalization()(x)
  x = Conv2D(256, 3, padding="same", activation="relu", 
             kernel_initializer=my_init(vgg.layers[8].weights[0]),
             bias_initializer=my_init(vgg.layers[8].weights[1]),
             kernel_regularizer=regularizers.l2(w_reg))(x)
  x = BatchNormalization()(x)
  x = Conv2D(256, 3, padding="same", activation="relu", 
             kernel_initializer=my_init(vgg.layers[9].weights[0]),
             bias_initializer=my_init(vgg.layers[9].weights[1]),
             kernel_regularizer=regularizers.l2(w_reg))(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D(2)(x)
  
  return x

  
def wide_resnet_f(x, w_reg):
  n = 2 # depth = 6*n + 4
  k = 4 # widen factor
  
  def bottleneck(x, out_filter, dropout=None, first_stride=2):
    for i in range(n):
      print(i,n, x.shape)
      shortcut = x
      
      # 1st conv
      x = Activation("relu")(BatchNormalization()(x))
      # only use shortcut if not bottleneck in a group
      if i == 0:
        shortcut = x
      # decrease dimension on very first conv
      x = Conv2D(out_filter, 3, strides=(first_stride if i == 0 else 1), 
                 padding="same", kernel_initializer="glorot_normal",
                 kernel_regularizer=regularizers.l2(w_reg))(x)
      
      # 2nd conv           
      x = Activation("relu")(BatchNormalization()(x))
      if dropout is not None:
        x = Dropout(dropout)(x)
      x = Conv2D(out_filter, 3, strides=1, 
                 padding="same", kernel_initializer="glorot_normal",
                 kernel_regularizer=regularizers.l2(w_reg))(x)
      
      if i == 0:
        shortcut = Conv2D(out_filter, 1, strides=first_stride, 
                          padding="same", kernel_initializer="glorot_normal",
                          kernel_regularizer=regularizers.l2(w_reg))(shortcut)
      x = Add()([shortcut, x])
      #x = merge([shortcut, x], mode="sum")
    return x
  
  x = Conv2D(16, 3, padding="same", kernel_initializer="glorot_normal",
             kernel_regularizer=regularizers.l2(w_reg))(x)
  
  x = bottleneck(x, 16*k, dropout=None, first_stride=1)
  x = bottleneck(x, 32*k, dropout=None)
  x = bottleneck(x, 64*k, dropout=None)
  x = Activation("relu")(BatchNormalization()(x))
  x = AveragePooling2D((8, 8), strides=(1, 1))(x)
  #x = Flatten()(x)
  # x = Dense(num_classes, 
            # activation="softmax",
            # kernel_initializer="glorot_normal", 
            # kernel_regularizer=regularizers.l2(w_reg))(x)

  return x#Model(inputs=input, outputs=x) 

In [0]:
from __future__ import print_function
from scipy.io import loadmat
from keras import backend as K
#K.clear_session()
import cv2 as cv
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
import os

all_data = loadmat("data2.mat")['X']



In [8]:
print(all_data.shape)
def get_train(all_data, sample_size):
    
    while True:
      x_matrix = np.zeros(shape=(sample_size, 11, 100, 100, 3))
      y_matrix = np.zeros(shape=(sample_size, 10, 1))
      
      for i in range(sample_size):
          training_subset_index = np.random.randint(901, size = 10)
          suspect_index = np.random.randint(0, 9, 1)
          training_subset = np.zeros((10,100,100,3))
          k = 0
          for j in range(10): 
            k = np.random.randint(low=2, size=1)
            training_subset[j, :, :, :] = all_data[training_subset_index[j], k[0],:, :, :].reshape((1, 100, 100, 3))
          l = np.random.randint(low = 3, size=1)
          while True:
            if l[0] != k[0]:
              break
            else:
              l = np.random.randint(low = 3, size=1)
          #print(l[0])
          #print(k[0])
          suspect = all_data[training_subset_index[suspect_index[0]], l[0], :, :, :].reshape((1, 100, 100, 3))

          #target_array.append(suspect_index[0])
          #
          # y_array = np.zeros(shape=(11, 1))
          y_array = np.zeros(shape=(10, 1))
          y_array[suspect_index] = np.array([1])
          #y_array[np.random.randint(0, 9, 1)[0]] = np.array([1])

          x = np.zeros(shape=(11, 100, 100, 3))

          x[0, :, :, :] = suspect
          x[1:, :, :, :] = training_subset

          x_matrix[i, :, :, :, :] = x
          y_matrix[i, :, :] = y_array

      yield x_matrix, y_matrix.reshape(-1,10)#, target_array

generator = get_train(all_data, 1000)
x_train, y_train = next(generator)
print("")
print(x_train.shape)
print(y_train.shape)
#print(target_array.shape)

(901, 3, 100, 100, 3)

(1000, 11, 100, 100, 3)
(1000, 10)


In [9]:
import numpy as np
test_index = 4
test_index_pos = np.argmax(y_train[test_index])
print(y_train[test_index])
print("Sanity check these two are different")
print(x_train[test_index,0,:,0,0])
print(x_train[test_index,1+test_index_pos,:,0,0])
print("Sanity check difference norm is not 0")
print(np.linalg.norm(x_train[test_index,0,:,0,0] - x_train[test_index,1+test_index_pos,:,0,0]))
print([np.linalg.norm(x_train[test_index, i] for i in range(11))])
#print(all_data[0,:,:,0,0])

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
Sanity check these two are different
[ 51.  69.  66.  63.  59.  59.  63.  68.  70.  69.  68.  63.  58.  56.
  58.  62.  67.  69.  68.  66.  60.  57.  56.  59.  62.  64.  60.  61.
  54.  47.  48.  52.  61.  67.  73.  76.  74.  72.  72.  67.  67.  71.
  77.  84. 105. 126. 141. 154. 164. 171. 172. 171. 172. 173. 175. 174.
 173. 171. 170. 167. 157. 145. 127. 105.  79.  50.  38.  40.  43.  48.
  53.  54.  56.  51.  48.  43.  43.  38.  43.  49.  52.  53.  49.  48.
  42.  38.  43.  45.  48.  56.  66.  65.  53.  46.  51.  56.  61.  58.
  64.  57.]
[ 4.  1.  3.  5.  8.  3.  3.  6.  9.  8.  6.  5. 10. 22. 34. 40. 40. 41.
 36. 32. 30. 19.  7.  5.  1.  1.  1.  4.  5.  4.  5.  8.  5.  3.  2.  4.
  5.  3.  1.  2.  5.  7.  7.  9. 13. 13. 12. 12. 14. 14. 17. 17. 19. 21.
 21. 21. 22. 22. 24. 25. 25. 26. 27. 30. 33. 33. 35. 36. 35. 35. 37. 37.
 37. 37. 37. 38. 40. 38. 36. 38. 39. 38. 36. 36. 36. 36. 36. 36. 37. 38.
 36. 36. 36. 36. 36. 35. 32. 32. 32. 32.]
Sanity check di

TypeError: ignored

In [10]:
batch_size = 20
# num_classes = 10
num_classes = 2
epochs = 5
#data_augmentation = True
#save_dir = os.path.join(os.getcwd(), 'saved_models')
#model_name = 'data2.h5'

# print(y_train)
# y_train = keras.utils.to_categorical(y_train, num_classes)
# print(y_train)



w_reg = 1e-4


model = build_model([11,100,100,3], 1e-4, 10, opt={
  "model": "vgg", 
  "fc_after_cnn": True,
  "fc_after_cnn_dim": 25,
  "comparing_method": "mmul",
  "comparing_method_mmul_w_dim": 5,
  "comparing_method_mmul_use_original_target": False,
  "": ""
})


CNN output size: 36864
Embedding size: 25
Doing mmul weights prediction with shape: [25, 5]
(?, 10)


In [0]:

opt = keras.optimizers.SGD(lr=0.01, decay=0.0, momentum=0.90, nesterov=False)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_train
x_train /= 255
x_test /= 255


def lrf(e):
  if e > 40: 
    return 1e-4
  if e > 30:
    return 1e-3
  if e > 15:
    return 1e-2
  return 1e-1

lrate = LearningRateScheduler(lrf)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.1,
                              patience=5, 
                              verbose=1,
                              min_lr=1e-5)

# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=True,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=True,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train.reshape(-1, 100, 100, 3))

#model.summary()



In [15]:
import sys as sys
sys.getsizeof(model)
model.fit(x_train, y_train, epochs = 1, batch_size= 25, verbose = True)
model.save('model.h5')
# dynamic
#generator = get_train(all_data, 10)
#model.fit_generator(generator, steps_per_epoch = 100, epochs=20, verbose = True)



#datagen.fit(x_train)



Epoch 1/1
1000/1000 [==============================] - 77s 77ms/step - loss: 2.3708 - acc: 0.0930


TypeError: ignored

In [20]:
from google.colab import files
uploaded = files.upload()

Saving brett.png to brett.png
Saving clark.jpg to clark.jpg
Saving leonard.jpg to leonard.jpg
Saving ming-na.jpg to ming-na.jpg
Saving sheldon.jpg to sheldon.jpg
Saving shield.jpg to shield.jpg
Saving tbbt.jpg to tbbt.jpg


In [24]:
from google.colab import files
uploaded = files.upload()

Saving haarcascade_frontalface_default.xml to haarcascade_frontalface_default.xml


In [2]:
import cv2 as cv
import numpy as np
# x_test = np.array([datagen.standardize(x) for x in x_test])


# Score trained model.

#y_test_pred_onehot = model.predict(x_test)
#y_test_pred = np.argmax(y_test_pred_onehot, axis=1)
#print(y_test_pred)
#print(y_test_pred_onehot)


def get_bounding_boxes(img):
    face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 3)
    return faces


#candidate = input("Candidate File Name: ")
#suspect = input("Suspect File Name: ")

candidate = 'leonard.jpg'
suspect = 'sheldon.jpg'

img1 = cv.imread(candidate,1)
img2 = cv.imread(suspect,1)
faces1 = get_bounding_boxes(img1)
faces2 = get_bounding_boxes(img2)

# gray1 = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
# for (x,y,w,h) in faces1:
#     cv.rectangle(img1,(x,y),(x+w,y+h),(255,0,0),2)
#     roi_gray = gray1[y:y + h, x:x + w]
#     print(roi_gray.shape)
#     roi_color = img1[y:y + h, x:x + w]
# cv.imshow('img', img1)
# cv.imwrite('two_people_identified.png', img1)
# cv.waitKey(0)
# cv.destroyAllWindows()

print(faces1.shape)
print(faces2.shape)
#%%

(5, 4)
(1, 4)


In [3]:
n = 10

X = np.random.uniform(low = 0.0, high = 1.0, size = (n+1,100,100,3))

for i in range(0,min(n,faces1.shape[0])+1):
    if i == 0:
        box = faces2[0,:]
        pic = img2[box[1]:(box[1]+box[3]), box[0]:(box[0]+box[2]),:]
        newPic = None
        dimx = pic.shape[0]
        dimy = pic.shape[1]
        newPic = cv.resize(pic, newPic, fx=100/dimx, fy=100/dimy)
        X[i, :, : ,:] = newPic / 255
    else:
        box = faces1[i-1,:]
        pic = img1[box[1]:(box[1]+box[3]), box[0]:(box[0]+box[2]),:]
        newPic = None
        dimx = pic.shape[0]
        dimy = pic.shape[1]
        newPic = cv.resize(pic, newPic, fx=100/dimx, fy=100/dimy)
        X[i, :, : ,:] = newPic / 255


for i in range(faces1.shape[0],10):
    X[i, :, :, :] = X[i - faces1.shape[0], :, :, :]

# X[7, :, : ,:] = X[4, :, : ,:]
# X[8, :, : ,:] = X[1, :, : ,:]
X[9, :, : ,:] = X[2, :, : ,:]
X[10, :, : ,:] = X[3, :, : ,:]
# #%%

##Model Stuff
p = model.predict(np.array([X]))
print(p)

#%%
i = p.argmax()
box = faces1[i,:]
cv.rectangle(img1, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]),(255,0,0),2)
cv.imshow('img',img1)
cv.waitKey(0)
cv.destroyAllWindows()

NameError: ignored